# MULTIVAC Pre-Cooked Output
## Introduction
Gallup's other Jupyter notebook, `precooked_replication.ipynb`, walked through the step-by-step -- with data inputs and outputs -- the various pieces to MULTIVAC. This notebook here takes the end product created by those steps (a Markov logic network or MLN), a knowledge graph and attributes, including semantic clustering, from the parsed articles as a pickle file called `mln.pkl` and runs the evaluation steps against the data to asses value and accuracy. 

## Query mapping
Once the MLN knowledge base has been compiled and test queries have been generated, the prototype MULTIVAC system provides a basic interface for mapping these queries to the knowledge base to retrieve answers.
 
The evaluation script is called from `USP.py` and takes three parameters determining the eval and results directories and the name of the file with queries to test. The directory parameters specify the locations of the query file and saved MLN data files, respectively; the eval directory is also where any resulting answers will be written out.
 
The system first reads in the MLN data, followed by the query file. Each question in the query file (one per line) is parsed using the Stanford CoreNLP parser engine, and a custom parse object is returned containing a list of token objects specifying form, lemma, POS tag, NER tag, as well as dependency and parent/child token relationships.
 
Once parsed, each question is analyzed, and key relationships and related arguments and dependencies are extracted. To begin, all non-auxiliary verbs with children are extracted as key relations to analyze; if no such verbs are present, the system picks the token identified as the root of the sentence, whatever it is. As this implies, a convoluted or complex question could indeed result in multiple "queries" being applied to the MULTIVAC knowledge base.
 
For each key relationship extracted, MULTIVAC assumes there is at least one "known" component and one "missing" component that the query is attempting to locate. MULTIVAC searches for all child tokens that are nouns or are tagged as `nsubj`, `nubjpass`, `dobj`, or `obj` dependents. If one matching `nsubj` or `nsubjpass` is found, the missing argument is determined to be of type `dobj` and `nsubj` if not. If the known argument itself has children, these are added to the argument as a compound phrase before matching against the knowledge base.
 
MULTIVAC maps a query to the MLN knowledge base by searching through the argument clusters contained within the key relation’s semantic cluster and finding matches to the known and missing argument types. For each set of argument type matches, the system attempts to find matching semantic clusters for the arguments, indicating a subnetwork within the MLN that is semantically isomorphic to the query and complete. If it finds such a match, it returns an answer filling in the missing information. Returned answers are compiled and printed out to a file in the eval directory called “Answers.txt.”
 
As of this date, there are still complications with returning answers to the generated test queries from the MLN knowledgebase. The MULTIVAC team continues to work to identify and resolve bugs and this prototype is continually evolving. Several considerations have impacted this effort, not least of which is the complexity involved in accepting an indefinitely broad range of potential questions and question types. The original system this interface is based on depended on receiving one of two types of rigorously formatted questions that were hand-curated.